In [1]:
import os
SNAPPY_notebook_path = os.path.join(os.path.abspath(""), "sunak.ipynb")

In [2]:
import time
SNAPPY_start_time = time.perf_counter_ns()

In [3]:
%load_ext autotime

time: 48.1 µs (started: 2024-05-24 23:38:40 -04:00)


In [4]:
%load_ext snappy_jupyter.snappy_magic

time: 462 ms (started: 2024-05-24 23:38:40 -04:00)


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import os
# STEFANOS: Conditionally import Modin Pandas
import pandas as pd


#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(os.path.abspath('') + '/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/home/sam/dias-benchmarks/dias-benchmarks/notebooks/mpwolke/just-you-wait-rishi-sunak/input/latest-elected-uk-prime-minister-rishi-sunak/uk_pm.parquet
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/mpwolke/just-you-wait-rishi-sunak/input/latest-elected-uk-prime-minister-rishi-sunak/uk_pm.csv
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/mpwolke/just-you-wait-rishi-sunak/input/latest-elected-uk-prime-minister-rishi-sunak/uk_pm.pkl
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/mpwolke/just-you-wait-rishi-sunak/input/latest-elected-uk-prime-minister-rishi-sunak/uk_pm.feather
time: 1.03 ms (started: 2024-05-24 23:38:40 -04:00)


In [6]:
import re
import string

import nltk
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# STEFANOS: Unneeded
# from wordcloud import WordCloud
import matplotlib.style as style
style.use('fivethirtyeight')

time: 407 ms (started: 2024-05-24 23:38:40 -04:00)


In [7]:
train = pd.read_parquet(os.path.abspath('') + '/input/latest-elected-uk-prime-minister-rishi-sunak/uk_pm.parquet')

time: 111 ms (started: 2024-05-24 23:38:41 -04:00)


In [8]:
train.head()

,text,username,hashtags,created_at,user followers count,replycount,retweetcount,likecount,quotecount,language,quotedtweet,inReplyToTweetId,inReplyToUser,mentionedUsers
id,,,,,,,,,,,,,,
1584696134693711873,@BritIndianVoice @RishiSunak #RishiSunak is #B...,IsmailYSyed,"['RishiSunak', 'BritishHindu', 'Pakistani', 'P...",2022-10-24 23:59:35+00:00,393,0,0,0,0,en,<NA>,1.584541e+18,https://twitter.com/BritIndianVoice,@BritIndianVoice @RishiSunak #RishiSunak is #B...
1584696057841098752,#RishiSunak is the richest Tory MP in history....,AdameMedia,['RishiSunak'],2022-10-24 23:59:17+00:00,9830,3,23,68,4,en,<NA>,NaN,<NA>,#RishiSunak is the richest Tory MP in history....
1584695906091536385,#RishiSunak,ROHITKU96224077,['RishiSunak'],2022-10-24 23:58:40+00:00,3,0,0,0,0,qht,<NA>,NaN,<NA>,#RishiSunak
1584695822658441217,A mountain full of money and a knowledge of pr...,Parisistas,['RishiSunak'],2022-10-24 23:58:21+00:00,0,0,0,0,0,en,<NA>,NaN,<NA>,A mountain full of money and a knowledge of pr...
1584695640696946688,@trussliz @RishiSunak Chance was been given as...,baboo_prasant,['RishiSunak'],2022-10-24 23:57:37+00:00,14,0,0,0,0,en,<NA>,1.584546e+18,https://twitter.com/trussliz,@trussliz @RishiSunak Chance was been given as...


time: 9.35 ms (started: 2024-05-24 23:38:41 -04:00)


In [9]:
df = pd.read_csv(os.path.abspath('') +"/input/latest-elected-uk-prime-minister-rishi-sunak/uk_pm.csv", delimiter=',', encoding='utf8')
pd.set_option('display.max_columns', None)
df.tail()

,id,text,username,hashtags,created_at,user followers count,replycount,retweetcount,likecount,quotecount,language,quotedtweet,inReplyToTweetId,inReplyToUser,mentionedUsers
113573,1584553348736421888,@JackBarbour_ @piersmorgan @RishiSunak The man...,aqeelone22,NaN,2022-10-24 14:32:12+00:00,1162,0,0,0,0,en,NaN,1.584532e+18,https://twitter.com/JackBarbour_,@JackBarbour_ @piersmorgan @RishiSunak The man...
113574,1584553348677525505,@Jacob_Rees_Mogg @RishiSunak Let's hope you're...,EddieThornley,NaN,2022-10-24 14:32:12+00:00,97,0,0,0,0,en,NaN,1.584539e+18,https://twitter.com/Jacob_Rees_Mogg,@Jacob_Rees_Mogg @RishiSunak Let's hope you're...
113575,1584553348597895168,@sajidjavid @RishiSunak Desperate call for a d...,pedleysjohn,['ToriesUnfitToGovern'],2022-10-24 14:32:12+00:00,2484,0,0,0,0,en,NaN,1.584401e+18,https://twitter.com/sajidjavid,@sajidjavid @RishiSunak Desperate call for a d...
113576,1584553348132397056,@SkyNews Rulling Party needs a strong oppositi...,KarobiVC,NaN,2022-10-24 14:32:12+00:00,392,0,0,0,0,en,NaN,1.584512e+18,https://twitter.com/SkyNews,@SkyNews Rulling Party needs a strong oppositi...
113577,1584553347998105600,@trussliz @RishiSunak Liz Truss has won the ma...,sands_hill,NaN,2022-10-24 14:32:12+00:00,4,0,0,0,0,en,NaN,1.584546e+18,https://twitter.com/trussliz,@trussliz @RishiSunak Liz Truss has won the ma...


time: 397 ms (started: 2024-05-24 23:38:41 -04:00)


In [10]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

print(len(train[train['likecount'] < 500]), 'tweets with less than 500 dislikes')
print(len(train[train['likecount'] > 500]), 'tweets with more than 500 dislikes')

113332 tweets with less than 500 dislikes
246 tweets with more than 500 dislikes
time: 19.8 ms (started: 2024-05-24 23:38:41 -04:00)


In [11]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

# video with the most comments

train[train['likecount'] == train['likecount'].max()]['text'].iloc[0]

"Warmest congratulations @RishiSunak! As you become UK PM, I look forward to working closely together on global issues, and implementing Roadmap 2030. Special Diwali wishes to the 'living bridge' of UK Indians, as we transform our historic ties into a modern partnership."

time: 2.01 ms (started: 2024-05-24 23:38:41 -04:00)


In [12]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

def remove_line_breaks(text):
    text = text.replace('\r', ' ').replace('\n', ' ')
    return text

#remove punctuation
def remove_punctuation(text):
    re_replacements = re.compile("__[A-Z]+__")  # such as __NAME__, __LINK__
    re_punctuation = re.compile("[%s]" % re.escape(string.punctuation))
    '''Escape all the characters in pattern except ASCII letters and numbers'''
    tokens = word_tokenize(text)
    tokens_zero_punctuation = []
    for token in tokens:
        if not re_replacements.match(token):
            token = re_punctuation.sub(" ", token)
        tokens_zero_punctuation.append(token)
    return ' '.join(tokens_zero_punctuation)

def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    return text

def lowercase(text):
    text_low = [token.lower() for token in word_tokenize(text)]
    return ' '.join(text_low)

def remove_stopwords(text):
    stop = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(text)
    text = " ".join([word for word in word_tokens if word not in stop])
    return text

#remobe one character words
def remove_one_character_words(text):
    '''Remove words from dataset that contain only 1 character'''
    text_high_use = [token for token in word_tokenize(text) if len(token)>1]      
    return ' '.join(text_high_use)   
    
#%%
# Stemming with 'Snowball stemmer" package
def stem(text):
    stemmer = nltk.stem.snowball.SnowballStemmer('english')
    text_stemmed = [stemmer.stem(token) for token in word_tokenize(text)]        
    return ' '.join(text_stemmed)

def lemma(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    word_tokens = nltk.word_tokenize(text)
    text_lemma = " ".join([wordnet_lemmatizer.lemmatize(word) for word in word_tokens])       
    return ' '.join(text_lemma)


#break sentences to individual word list
def sentence_word(text):
    word_tokens = nltk.word_tokenize(text)
    return word_tokens
#break paragraphs to sentence token 
def paragraph_sentence(text):
    sent_token = nltk.sent_tokenize(text)
    return sent_token    


def tokenize(text):
    """Return a list of words in a text."""
    return re.findall(r'\w+', text)

def remove_numbers(text):
    no_nums = re.sub(r'\d+', '', text)
    return ''.join(no_nums)



def clean_text(text):
    _steps = [
    remove_line_breaks,
    remove_one_character_words,
    remove_special_characters,
    lowercase,
    remove_punctuation,
    remove_stopwords,
    stem,
    remove_numbers
]
    for step in _steps:
        text=step(text)
    return text   
#%%

time: 1.77 ms (started: 2024-05-24 23:38:41 -04:00)


In [13]:
#https://stackoverflow.com/questions/55557004/getting-attributeerror-float-object-has-no-attribute-replace-error-while
#To avoid with tqdm AttributeError: 'float' object has no attribute

train["text"] = train["text"].astype(str)
train["text"] = [x.replace(':',' ') for x in train["text"]]

time: 160 ms (started: 2024-05-24 23:38:41 -04:00)


In [ ]:
# %%snappy
def extracted_cell(clean_text: Callable[()], train: DataFrame) -> DataFrame:
    """
Params:
clean_text: typing.Callable
train: Rows: Single-level Index: Integer
       Columns: Single-level Index: ['text: string', 'username: unknown', 'hashtags: unknown', 'created_at: unknown', 'user followers count: integer', 'replycount: integer', 'retweetcount: integer', 'likecount: integer', 'quotecount: integer', 'language: unknown', 'quotedtweet: unknown', 'inReplyToTweetId: float', 'inReplyToUser: unknown', 'mentionedUsers: unknown']

Returns:
Rows: Single-level Index: Integer
Columns: Single-level Index: ['text: string', 'username: unknown', 'hashtags: unknown', 'created_at: unknown', 'user followers count: integer', 'replycount: integer', 'retweetcount: integer', 'likecount: integer', 'quotecount: integer', 'language: unknown', 'quotedtweet: unknown', 'inReplyToTweetId: float', 'inReplyToUser: unknown', 'mentionedUsers: unknown', 'clean_text: string']
"""
    train['clean_text'] = pd.Series([clean_text(i) for i in train['text']])
    return train
train = extracted_cell(clean_text, train)

* Original output:
```

```

In [ ]:
words = train["clean_text"].values

In [ ]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

ls = []

for i in words:
    ls.append(str(i))

In [ ]:
ls[:5]

In [ ]:
# STEFANOS: Disable plotting
# #Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

# # The wordcloud 
# plt.figure(figsize=(16,13))
# wc = WordCloud(background_color="lightblue", colormap='Set2', max_words=1000, max_font_size= 200,  width=1600, height=800)
# wc.generate(" ".join(ls))
# plt.title("Most discussed terms", fontsize=20)
# plt.imshow(wc.recolor( colormap= 'Set2' , random_state=17), alpha=0.98, interpolation="bilinear", )
# plt.axis('off')

In [ ]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

most_pop = train.sort_values('likecount', ascending =False)[['text', 'likecount']].head(12)

most_pop['target1'] = most_pop['likecount']/1000

In [ ]:
# STEFANOS: Disable plotting

# #Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

# plt.figure(figsize = (30,35))

# sns.barplot(data = most_pop, y = 'text', x = 'target1', color = 'c')
# plt.xticks(fontsize=50, rotation=0)
# plt.yticks(fontsize=50, rotation=0)
# plt.xlabel('Votes in Thousands', fontsize = 21)
# plt.ylabel('')
# plt.title('Most popular tweets', fontsize = 50);

In [ ]:
import collections.abc
#gensim aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk

In [ ]:
stemmer = SnowballStemmer('english')

In [ ]:
# STEFANOS: Disable. Connects to net.
# nltk.download('wordnet')

In [ ]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
train['text'].iloc[2]

In [ ]:
import nltk
# STEFANOS: Disable. Connects to net.
# nltk.download('omw-1.4')

In [ ]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

doc_sample = train['text'].iloc[1]
print('original document: ')

words = []

for word in doc_sample.split(' '):
    words.append(word)
    
    
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

In [ ]:
train['text'] = train['text'].astype(str)

In [ ]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

words = []

for i in train['text']:
        words.append(i.split(' '))

In [ ]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

dictionary = gensim.corpora.Dictionary(words)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in words]
bow_corpus[4310]

In [ ]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction

bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
SNAPPY_end_time = time.perf_counter_ns()
print("Total elapsed time:", (SNAPPY_end_time - SNAPPY_start_time) / (10 ** 9))